In [70]:
# Part 1
Y = 0
X = 1


def parse(input_str):
    return [list(line) for line in input_str.splitlines()]


def n(yx):
    if yx[Y] == 0:
        return None

    y = yx[Y] - 1
    x = yx[X]
    if grid[y][x] == "#":
        return None

    return (y, x)


def e(yx):
    if yx[X] == WIDTH - 1:
        return None

    y = yx[Y]
    x = yx[X] + 1
    if grid[y][x] == "#":
        return None

    return (y, x)


def s(yx):
    if yx[Y] == HEIGHT - 1:
        return None

    y = yx[Y] + 1
    x = yx[X]
    if grid[y][x] == "#":
        return None

    return (y, x)


def w(yx):
    if yx[X] == 0:
        return None

    y = yx[Y]
    x = yx[X] - 1
    if grid[y][x] == "#":
        return None

    return (y, x)


def neighbours(cell):
    candidates = [n(cell), e(cell), s(cell), w(cell)]
    return [c for c in candidates if c]


def find_start():
    for y in range(HEIGHT):
        for x in range(WIDTH):
            if grid[y][x] == "S":
                return (y, x)

    raise Exception("No start found")


def dump_grid(grid, visited=set()):
    for y in range(len(grid)):
        for x in range(len(grid[y])):
            if (y, x) in visited:
                print("O", end="")
            else:
                print(grid[y][x], end="")
        print("")


# Go
grid = parse(input_str)
HEIGHT = len(grid)
WIDTH = len(grid[0])

steps = 64

frontier = set([find_start()])

for i in range(steps):
    next_frontier = set()
    for cell in frontier:
        for next in neighbours(cell):
            next_frontier.add(next)

    frontier = next_frontier
    # print(f"After {i + 1} steps, visited {len(frontier)} cells")

print("Part 1:", len(frontier))

Part 1: 3733


In [67]:
# Test Input
input_str="""
...........
.....###.#.
.###.##..#.
..#.#...#..
....#.#....
.##..S####.
.##..#...#.
.......##..
.##.#.####.
.##..##.##.
...........
""".strip()


In [69]:
# Input
input_str="""
...................................................................................................................................
.#...............#..#...........#....#................................................##......##..........#..............#.........
......#...................#...#..#..........#.#.......#.....................#.............##.....#....#......#..#........#.......#.
.#....#...#...#..#.................#.....#....#.#..#..........................#.....#...........#.............#....................
.#.............................##...##......#........#......................#.........##...#..#.................##.........#.#.....
....#.#....#.................#..#..#........#...#.#.#..#......................#........#....#..............#.....##................
....#.....#......#....#.##....#.##........#.....#......#........................#..........#.............#...................#.....
...........#...........##.#..........##.....#...#...##........#.................#.......#.............#......#..#..........#.......
..#...........#.............#...................................#.....................##...........#..............#.#....#.........
....#.....#.........#....#......##..............#...#........#.##....#........#..#....#..........#...................#..........#..
............#.........#............#.#...........#.........#.........................#......#..#......#..#..#.......#.....#........
.........#.................#........#..#.....#.............#........................#..#...#...#..#..#.....#...#..#......#....#....
..........#.#.#...............#.........#....##.#..........##......##...............#...#...###.##...#.......#..............#......
....#.........#...#........#.##.............#............#........#....##......................#..........#......#...#.......#.....
.....##..#......#...#........#................#............................#..........#............##..#...#.....#..#........##....
..#........#.#.###.....#....#..#.....#.....#..........................#.....#...........#.##..........#..........#...#...#.........
...................#.................................#.....#....#...#...#...................#..#......#....#.#.##................#.
.........#...#..#.#....#........#..........#................#.#....#...#.............................#..................#..#.#.....
...........#...#.#...................#..........................#..........#...................#...............#.###........#..#...
......#.......#...............#.........#.#...........#.####............#...#............#...##..##..........#............#........
..............................#..#.###..................#.....#......#..#.#..#...#..............#...#.....#.##..#.............#....
...............#.......#.#....#......#.#..........#..#.................#.#........#.......#.................#...............#...##.
............#.................#....................#............#....#.....#...............#........###........#.#..#.##...#.......
...#.....#.#..#.....#....#...#......#...........................#..#..#.......................#..............#..........#..........
.#....##..............#.##....#...#................#............#..........#......##....................##....#.#..#......#..##....
.....#.........#...##.#.#......................#....#...#...#.............#...................#...#..#...#...###......#...#....#...
........##......#.....#......................................#....###.#...#..........#.............#.....#.....#..........#.#......
........##.#...##.........##.....................#.#...........#........#.......................#...#.....#..............#.#.......
..........#.##.#..#..#..#.......#.....................................#.......##....#................#....#.....#...##.#....#..##..
..##.......#.........#..................##........#.....#.....................#.....#...................#.....#.#.........#........
....#........#.....#...................#.#..#..........................#...........#.............................#......#....#.....
..........................................#............#..#.......................#...#.#..#..................#.#............##....
...#..#....#......#....................................................................#..............#.#................#.......#.
..#.....##.............#.#.#...........#........#.................#..................#....................#..............#..###....
...##..#............#...#..#.........#..#.....#...........................#.....#....#...#...............#....#.............#...#..
...........#.......#...............................#..#.#...#.#...#...#........#.............##......................#...........#.
...............#..................#...............#.............#..#......................#......................##..........#.....
...........#..#....#.##...........#....#.##.#..........#...#.........#..........#.#..............................##.........#..#.#.
......#...#.#..##..................##................#####.#........#....#..#........#....#.....#................##..........#.....
....#.#........#.....#.......................##.......................#..#.#...........#.........................#..#..............
...#...........#..............#.......#.#....#......#...#....#.....#.....#...............#...........................#......#....#.
....#...........#..#.................#...##....#.............#..................#...........#.#.#............................#.....
........#..................#..#.#.........................###...#..#.#.#..##......#........##..#.....#..........#.....#.........#..
.....#.#.#....#...#..............#.#..#.......#.........###..#..........#.......#..#....................#....................##....
........#.......#...............##................#..............................#...#.#.....#....#.....##........#.#......#.......
.....#.##.....................#....#......###..#...........#........#....#..........#...#....#.........................#.......#...
..#.....#...#................#...##.#.......#....#.....#...............#.#....................#...#.......#..........#.............
........#.............#.#................#..#.....#....#.#......................#.....#.................##..#......................
..#..........#..........#..............#.........#..#.........#..............#.#.....#......###...#.....................#..........
......................#...#.#.#..#........#....###......##.........#.............#........#..#...#........#....................#...
..#..#...................#...#..#..#....#..##............#...#.............#.....#....##...............#...................##..#...
...#................###...............#...........#..............................#...#.........#.....#..#..#.................#...#.
....#.#..................##....#...#...#.........#....#.......#......#..........##..#..#..##.........#.....................#.......
......#..........#....#.....#.#.....#...#..........#.#......##.#......#............#.....#.....###.......#.#.#..#..................
...#.#.........##........#...#.....#......#...#.......#..................................#.........................................
..............#........................#.......###..#.......#.......#.#.......#....#.............#.......#...#.....#...............
.#..#..........#...........##..............#...#.........#.....##.#....................................#.......................#...
...................#........#....#.#...........#............#.#.#.##...##............#.#.....................#.....................
...........#....#........#...........#...#..##....#.#........#..#......#..##....................#...#..............................
.............#..........#.....#.......##......#......##.....#......##............................##.........#.........#.#..........
..............#.#...#.....#.....##..............#.......#.#..#.#......#...................#....#........#..........................
.....................##..#.#.....#..........#......#......#..#....#.#.............#...#......#..#..................................
...............#.#.##........#.........#..........#.....#............#...#.......##.........#........##....#..#....................
...........##....#.............#.................#...#..#...#........#...#.#..#..#.................#...#.....#.....................
..................................#........#..........#...........##.#.##....#.#..#............#.#.............#...................
.................................................................S.................................................................
......#..................###.##........#..#.....#....................#............#....#.#...#.#....#.###......#....##.......#.....
......................#.##......#.#......#..#....#.....................#...........#.....#..............#.#..#......#..............
..............................#..#............#......##.#.........#........#........#........................#.....................
..........#........#..........##.#.#..#...................#.#.#............#...............#...#.......#....#...##.................
............#.....#..........#.#.......#.#.........#....#.#.....#....#..........#.......#..............#........#..#....##.........
............#.............................#.................#......#..................#..#.....#..#...............#...##...........
.................###....#...#.#..............#.#..............#.......#...##....#.....#............#...#..##.##........#.......#.#.
....................#..#..##.#.#..##.....##...............#........#......##................#......................................
...................#.......#....#....#.......#..#.............#..........#.#..........#.##.............#...........................
....................................#..#...#....#......##..#.......#...#................#.....#....#.#.......#...#.#...............
.##...##.......#........#...#...#......#...#..............#..............##..##...#.........#...##...#.....................#..#....
.#....#.............#.#....#....#...............#..#....#..........#.......#......#.....#.................#.....................#..
.#............................#......##......#..#.#........#.....................#.#..#..##..#......#....................#..#......
.......#............#....#....#...#..#........#...............#......#.......#..................#......#...#.............#.........
..#.................#.####..#...........#...........#..............#..#.##..##....#................#.....#.................#...#...
.....#..............#...#........#................#....##.##..#..............#..##......#..#.........#................##...#...#...
.........##..........#.......#..........#.#....#......#...........###...........#.......................#..#.#................#.#..
.......#......#.......#......................................#..............#.................#..#........#..........#..........#..
.......#...#..............#.......##...........#.#.#......#.........#.......#....#........#..#..#........#.................#.......
.#.....##.#.#.................#.###...................##......#....##..##................#...............#.........................
..........##..#............#.#............#.......#..#.............#.......##.....##..#...##...#...................#...##.#...#....
.##...#.....##.............#.........#...............#.............................#...........#....................#.....##.......
..........##.#.............#............#...#..#....#...........#.....#..........#...#.......#..#.#................................
..#.....................................#.....................#......#...............#...#........#......................#.##......
.........#.#....#....#...............#......#...#........##...........#.......#....#....#...#......#.............#.....#....##.....
.............#....................#...........#...................#.#...................#.#..#....................#..........#.....
.....#.............#..#...................#................#......#...#.......#......#....#..##...................#.#.##.....#..#..
.#..##...#........#......................#...##.#..##.........................#............#.#............#..#.....................
.#...................#................#..............##...........#........#.#.....#.#...#.................#...##...#....#..#......
....#.......#..#...#.#...#.............#..#.##...#....#....#.......................#......#....#........#..#.......................
........###.......##..................#.......#....#.##....#..................##..........................#..#........#.........#..
.##.......##.....#.........#..........#......#...............#........#.#..#..........#...............#.#...#..........#...........
.#...##............#.........#..........##.....#..#......#......#....##...#...##.....#..#..............#.#.......##....#.#.#...#.#.
...........#...........................#..........#.#..........#....#..........#....................#.....##.............#.......#.
.........#..................##............#..................#................#.#.......#.............###..#....#.......#........#.
.......#...............#................#........#.....#.................##......#.......................#...............#.........
...##............#......#....................#...##.............#......#....#.....##.........................#.............#.......
.#.......#.#.#..#.......#......#..............#...#.........#......##.##.....#.....................#.....#.....#......##......#....
.#...................#.#....#.....#..........#..#...#.......................#...#....#.....................#....#...###.....#......
......#.....#......#....#.#.................#.........#.##...............##...................#..#......#..#.............#.........
..#........#....#.#............................................#......#...#.......#.....................#...................#......
.........#...#.#.....................................#....#..........................................##...#........................
..#...............##.#............#....#........##.#....#..................#...#...........#...#..#..#....#..........#....#....#...
............#.#.......###.......##.##.....................#...#.#...#..#....#...##............#.#.#..............#..#.#...#........
................................#..#.#....................#..#.#......#..###...............#..#...#...##.#.......#.......#.........
....#........#.......#..#..........##.........................#....#......##..#................#......#..#......#.#......#.#.......
.#..............#.#....#..........#.#....................#......#............................#....#......##...#.#.......#..........
....#.....##...#........#..#...........#...#.........#...............#.................#.#..#.................#..............#.....
...........................................##.............#.#.....#......#................#...#........#.......#..#.#..............
......#.....#..#.......##....#..........#......................#............#..........#...#...............#..#..#....#...#...#....
..........#...##.....................#.................##...............##.............#..##...........#.#............#.....##.....
.#.#........................#...#.....#...........................#........................#........#.....#.#.#...........#........
..#...#....................#..#...#.........#...#........#.#...#...................#..#.#.....................#...........#........
...#.#.......#...#.................#......#...#...#....................##.................#..........##.......#..#.....#........#..
..................#.......#................#...............#.#....#....#......................#.....#.......#.........##...#.......
..#.......#.#..............#..###........##.#.................................##..........#........#..........#.....#.....#........
.........##.....#.........#..........#...........#.#.................#.........#..............#...............#......#.#...##......
...............#.......#.#...#...........#.....#...#........................#..............#.......#.#......#....#......#....#.....
.................##.#.#........##..........................................#....#.........##......#.......#..#..................#..
...#..##......#.....#.............##.#..##....#...............................##......###.#....###.#.........#...........#...#.##..
..#..#..............#.....#.....#..#..##.......#..##.....#.................#...#................#..#........#...##...#.............
.#..................#............#.#..................................................#......#.#.......###...........#........##.#.
............#..........#....#.....#...#.....#........#.#.........................#.#....#....#.##..........#......#................
....#....#..#....................#.....#........##......#...#.........##.....#...##...#...##.##..............#....#......#..#...#..
...................................................................................................................................
""".strip()